CSCE 4143: Data Mining

Group 7 Final Project

UFC Stuff

https://www.kaggle.com/datasets/akshaysinghim/ufc-fight-data-1993-to-2023

Levi Crider, Spencer Smith, Caleb Holmes, and Ethan Weems

## Installing Dependencies

In [270]:
import pandas as pd
import numpy as np
# load data
df = pd.read_csv("ufc_data.csv")

In [271]:
# preview dataset
pd.set_option('display.max_columns', None)
df

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,R_TD,B_TD,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,R_REV,B_REV,R_CTRL,B_CTRL,R_HEAD,B_HEAD,R_BODY,B_BODY,R_LEG,B_LEG,R_DISTANCE,B_DISTANCE,R_CLINCH,B_CLINCH,R_GROUND,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Aljamain Sterling,Sean O'Malley,0,1,17 of 35,25 of 35,48%,71%,24 of 46,26 of 36,0 of 2,0 of 0,0%,---,0,0,0,0,0:34,0:18,3 of 12,18 of 25,3 of 10,6 of 9,11 of 13,1 of 1,16 of 33,10 of 16,1 of 2,1 of 1,0 of 0,14 of 18,KO/TKO,2,0:51,5 Rnd (5-5-5-5-5),Marc Goddard,"August 19, 2023","Boston, Massachusetts, USA",UFC Bantamweight Title Bout,Sean O'Malley
1,Chris Weidman,Brad Tavares,0,0,37 of 92,70 of 115,40%,60%,39 of 94,71 of 116,0 of 8,0 of 0,0%,---,0,0,0,0,1:21,0:00,30 of 77,21 of 61,5 of 12,8 of 9,2 of 3,41 of 45,35 of 90,68 of 113,2 of 2,2 of 2,0 of 0,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,"August 19, 2023","Boston, Massachusetts, USA",Middleweight Bout,Brad Tavares
2,Neil Magny,Ian Garry,0,0,27 of 66,91 of 150,40%,60%,51 of 93,111 of 171,0 of 2,2 of 2,0%,100%,0,0,0,0,1:10,2:32,10 of 41,32 of 90,7 of 11,16 of 17,10 of 14,43 of 43,25 of 61,84 of 140,2 of 5,4 of 5,0 of 0,3 of 5,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,"August 19, 2023","Boston, Massachusetts, USA",Welterweight Bout,Ian Garry
3,Marlon Vera,Pedro Munhoz,0,0,141 of 251,113 of 246,56%,45%,141 of 251,113 of 246,0 of 1,0 of 1,0%,0%,0,0,0,0,0:00,0:00,109 of 212,53 of 177,9 of 14,20 of 28,23 of 25,40 of 41,138 of 246,111 of 244,3 of 4,2 of 2,0 of 1,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Herb Dean,"August 19, 2023","Boston, Massachusetts, USA",Bantamweight Bout,Marlon Vera
4,Zhang Weili,Amanda Lemos,1,0,163 of 217,24 of 63,75%,38%,296 of 358,29 of 68,6 of 7,0 of 0,85%,---,0,2,0,0,16:07,0:34,106 of 150,19 of 57,37 of 42,3 of 4,20 of 25,2 of 2,50 of 77,23 of 61,17 of 18,1 of 2,96 of 122,0 of 0,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Kevin MacDonald,"August 19, 2023","Boston, Massachusetts, USA",UFC Women's Strawweight Title Bout,Zhang Weili
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7267,Patrick Smith,Ray Wizard,0,0,1 of 1,1 of 1,100%,100%,1 of 1,2 of 2,0 of 1,0 of 0,0%,---,1,0,0,0,--,--,0 of 0,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0,0 of 0,Submission,1,0:58,No Time Limit,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Open Weight Bout,Patrick Smith
7268,Orlando Wiet,Robert Lucarelli,0,0,8 of 12,2 of 6,66%,33%,11 of 15,2 of 6,0 of 0,1 of 1,---,100%,0,1,0,0,--,--,7 of 11,1 of 2,1 of 1,0 of 1,0 of 0,1 of 3,1 of 3,2 of 6,0 of 0,0 of 0,7 of 9,0 of 0,KO/TKO,1,2:50,No Time Limit,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Open Weight Bout,Orlando Wiet
7269,Frank Hamaker,Thaddeus Luster,0,0,2 of 3,0 of 0,66%,---,14 of 15,0 of 0,1 of 1,0 of 1,100%,0%,3,0,1,0,--,--,2 of 3,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,1 of 1,0 of 0,0 of 0,0 of 0,1 of 2,0 of 0,Submission,1,4:52,No Time Limit,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Open Weight Bout,Frank Hamaker
7270,Johnny Rhodes,David Levicki,0,0,11 of 17,4 of 5,64%,80%,74 of 86,95 of 102,1 of 1,0 of 0,100%,---,0,0,0,0,--,--,9 of 15,4 of 5,1 of 1,0 of 0,1 of 1,0 of 0,1 of 1,1 of 2,1 of 1,2 of 2,9 of 15,1 of 1,KO/TKO,1,12:13,No Time Limit,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Open Weight Bout,Johnny Rhodes


In [272]:
df = df.rename(columns={'R_fighter': 'R_FIGHTER'})
df = df.rename(columns={'B_fighter': 'B_FIGHTER'})
df = df.rename(columns={'R_SIG_STR.': 'R_SIG_STR'})
df = df.rename(columns={'B_SIG_STR.': 'B_SIG_STR'})
df = df.rename(columns={'R_TOTAL_STR.': 'R_TOT_STR'})
df = df.rename(columns={'B_TOTAL_STR.': 'B_TOT_STR'})
df = df.rename(columns={'R_SIG_STR_pct': 'R_SIG_PCT'})
df = df.rename(columns={'B_SIG_STR_pct': 'B_SIG_PCT'})
df = df.rename(columns={'R_TD_pct': 'R_TD_PCT'})
df = df.rename(columns={'B_TD_pct': 'B_TD_PCT'})
df = df.rename(columns={'win_by': 'WIN_TYPE'})
df = df.rename(columns={'last_round': 'LAST_RD'})
df = df.rename(columns={'last_round_time': 'LAST_RD_TIME'})
df = df.rename(columns={'Format': 'NUM_RD'})
df = df.rename(columns={'Referee': 'REF'})
df = df.rename(columns={'date': 'DATE'})
df = df.rename(columns={'location': 'LOC'})
df = df.rename(columns={'Fight_type': 'TYPE'})
df = df.rename(columns={'Winner': 'WINNER'})

# Cleaning Data

In [273]:
# dropping all rows with missing values
attsWithPct = ["R_SIG_PCT", "B_SIG_PCT", "R_TD_PCT", "B_TD_PCT"]

for attribute in df.columns:
    if attribute in attsWithPct:
        continue
    else:
        df[attribute].replace("---", pd.NaT, inplace=True)
        df[attribute].dropna(inplace=True)

# dropping all rows before May of 2001
df['DATE'] = pd.to_datetime(df['DATE'], format="%B %d, %Y").dt.strftime("%Y-%m-%d")
limit_date = '2001-04-01'
df = df[(df['DATE'] > limit_date)]

In [274]:
# remove percentage signs from columns
df = df.replace('%', '', regex=True)

In [275]:
# combine last round and last round time into one column called total_time

# convert LAST_RD to time elapsed in rounds before final round
df['LAST_RD'] = (df['LAST_RD'].astype(int) * 5) - 5
df['LAST_RD'] = pd.to_timedelta(df['LAST_RD'], unit='m')

# convert LAST_RD_TIME to format acceptable by timedelta
df['LAST_RD_TIME'] = '00:' + df['LAST_RD_TIME'].astype(str)
df['LAST_RD_TIME'] = pd.to_timedelta(df['LAST_RD_TIME'])

# add times
df['TOTAL_TIME'] = df['LAST_RD'] + df['LAST_RD_TIME']

# reformat
# df['TOTAL_TIME'] = df['TOTAL_TIME'].apply(lambda x: '{:02}:{:02}'.format(x.components.minutes, x.components.seconds))
df['TOTAL_TIME'] = df['TOTAL_TIME'].dt.total_seconds()

df['R_CTRL'] = '00:' + df['R_CTRL'].astype(str)
df['R_CTRL'] = pd.to_timedelta(df['R_CTRL'])
df['R_CTRL'] = df['R_CTRL'].dt.total_seconds()

df['B_CTRL'] = '00:' + df['B_CTRL'].astype(str)
df['B_CTRL'] = pd.to_timedelta(df['B_CTRL'])
df['B_CTRL'] = df['B_CTRL'].dt.total_seconds()

# drop old columns
df = df.drop(columns=['LAST_RD', 'LAST_RD_TIME'])

In [276]:
df["NUM_RD"] = df["NUM_RD"].str[:1]

In [277]:
df

,R_FIGHTER,B_FIGHTER,R_KD,B_KD,R_SIG_STR,B_SIG_STR,R_SIG_PCT,B_SIG_PCT,R_TOT_STR,B_TOT_STR,R_TD,B_TD,R_TD_PCT,B_TD_PCT,R_SUB_ATT,B_SUB_ATT,R_REV,B_REV,R_CTRL,B_CTRL,R_HEAD,B_HEAD,R_BODY,B_BODY,R_LEG,B_LEG,R_DISTANCE,B_DISTANCE,R_CLINCH,B_CLINCH,R_GROUND,B_GROUND,WIN_TYPE,NUM_RD,REF,DATE,LOC,TYPE,WINNER,TOTAL_TIME
0,Aljamain Sterling,Sean O'Malley,0,1,17 of 35,25 of 35,48,71,24 of 46,26 of 36,0 of 2,0 of 0,0,---,0,0,0,0,34.0,18.0,3 of 12,18 of 25,3 of 10,6 of 9,11 of 13,1 of 1,16 of 33,10 of 16,1 of 2,1 of 1,0 of 0,14 of 18,KO/TKO,5,Marc Goddard,2023-08-19,"Boston, Massachusetts, USA",UFC Bantamweight Title Bout,Sean O'Malley,351.0
1,Chris Weidman,Brad Tavares,0,0,37 of 92,70 of 115,40,60,39 of 94,71 of 116,0 of 8,0 of 0,0,---,0,0,0,0,81.0,0.0,30 of 77,21 of 61,5 of 12,8 of 9,2 of 3,41 of 45,35 of 90,68 of 113,2 of 2,2 of 2,0 of 0,0 of 0,Decision - Unanimous,3,Keith Peterson,2023-08-19,"Boston, Massachusetts, USA",Middleweight Bout,Brad Tavares,900.0
2,Neil Magny,Ian Garry,0,0,27 of 66,91 of 150,40,60,51 of 93,111 of 171,0 of 2,2 of 2,0,100,0,0,0,0,70.0,152.0,10 of 41,32 of 90,7 of 11,16 of 17,10 of 14,43 of 43,25 of 61,84 of 140,2 of 5,4 of 5,0 of 0,3 of 5,Decision - Unanimous,3,Keith Peterson,2023-08-19,"Boston, Massachusetts, USA",Welterweight Bout,Ian Garry,900.0
3,Marlon Vera,Pedro Munhoz,0,0,141 of 251,113 of 246,56,45,141 of 251,113 of 246,0 of 1,0 of 1,0,0,0,0,0,0,0.0,0.0,109 of 212,53 of 177,9 of 14,20 of 28,23 of 25,40 of 41,138 of 246,111 of 244,3 of 4,2 of 2,0 of 1,0 of 0,Decision - Unanimous,3,Herb Dean,2023-08-19,"Boston, Massachusetts, USA",Bantamweight Bout,Marlon Vera,900.0
4,Zhang Weili,Amanda Lemos,1,0,163 of 217,24 of 63,75,38,296 of 358,29 of 68,6 of 7,0 of 0,85,---,0,2,0,0,967.0,34.0,106 of 150,19 of 57,37 of 42,3 of 4,20 of 25,2 of 2,50 of 77,23 of 61,17 of 18,1 of 2,96 of 122,0 of 0,Decision - Unanimous,5,Kevin MacDonald,2023-08-19,"Boston, Massachusetts, USA",UFC Women's Strawweight Title Bout,Zhang Weili,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7013,Tony DeSouza,Steve Berger,0,0,28 of 48,2 of 18,58,11,51 of 71,108 of 129,5 of 5,0 of 0,100,---,0,3,0,1,599.0,67.0,27 of 47,2 of 18,1 of 1,0 of 0,0 of 0,0 of 0,1 of 6,2 of 17,2 of 3,0 of 1,25 of 39,0 of 0,Decision - Unanimous,3,Mario Yamasaki,2001-05-04,"Atlantic City, New Jersey, USA",Welterweight Bout,Tony DeSouza,900.0
7014,BJ Penn,Joey Gilbert,0,0,22 of 32,3 of 7,68,42,76 of 86,11 of 15,2 of 2,0 of 1,100,0,0,0,0,0,270.0,6.0,18 of 28,0 of 4,4 of 4,2 of 2,0 of 0,1 of 1,3 of 8,1 of 5,3 of 3,2 of 2,16 of 21,0 of 0,KO/TKO,3,John McCarthy,2001-05-04,"Atlantic City, New Jersey, USA",Lightweight Bout,BJ Penn,297.0
7015,Shonie Carter,Matt Serra,2,0,18 of 33,10 of 25,54,40,49 of 65,27 of 44,2 of 3,2 of 13,66,15,0,3,2,1,363.0,271.0,6 of 17,5 of 18,6 of 7,2 of 2,6 of 9,3 of 5,10 of 16,5 of 19,6 of 6,3 of 4,2 of 11,2 of 2,KO/TKO,3,Mario Yamasaki,2001-05-04,"Atlantic City, New Jersey, USA",Welterweight Bout,Shonie Carter,891.0
7016,Semmy Schilt,Pete Williams,2,0,15 of 26,6 of 9,57,66,22 of 34,14 of 18,0 of 0,1 of 4,---,25,0,0,0,0,22.0,287.0,5 of 14,6 of 8,8 of 9,0 of 0,2 of 3,0 of 1,11 of 20,1 of 3,2 of 4,0 of 0,2 of 2,5 of 6,KO/TKO,3,John McCarthy,2001-05-04,"Atlantic City, New Jersey, USA",Heavyweight Bout,Semmy Schilt,388.0


In [278]:
def makeTwoCols(dataFrame, origCol):
    dataFrame[[origCol + "_SUC", origCol + "_ATT"]] = df[origCol].str.split(' of ', expand=True)
    dataFrame.drop(columns=[origCol], inplace=True)

In [279]:
attsToSplit = ["R_SIG_STR", "B_SIG_STR", "R_TOT_STR", "B_TOT_STR", "R_TD", "B_TD", "R_HEAD", "B_HEAD", "R_BODY", "B_BODY", 
               "R_LEG", "B_LEG", "R_DISTANCE", "B_DISTANCE", "R_CLINCH", "B_CLINCH", "R_GROUND", "B_GROUND"]

for attribute in attsToSplit:
    makeTwoCols(dataFrame=df, origCol=attribute)

In [280]:
R_subset = ['R_FIGHTER','R_KD','R_SIG_PCT','R_TD_PCT', 'R_SUB_ATT', 'R_GROUND_ATT', 'WIN_TYPE', 'NUM_RD', 'TYPE',
                'R_SIG_STR_SUC', 'R_SIG_STR_ATT', 'R_TOT_STR_SUC', 'R_TOT_STR_ATT', 'R_TD_SUC', 'R_TD_ATT',
                'R_GROUND_SUC', 'TOTAL_TIME','WINNER', 'LOC', 'REF', 'DATE']
R_df = df[R_subset]
R_df = R_df.rename(columns=lambda x: x.replace('R_', ''))
R_df = R_df.rename(columns={'SIG_STSUC':'SIG_STR_SUC'})
R_df = R_df.rename(columns={'SIG_STATT':'SIG_STR_ATT'})
R_df = R_df.rename(columns={'TOT_STSUC':'TOT_STR_SUC'})
R_df = R_df.rename(columns={'TOT_STATT':'TOT_STR_ATT'})
R_df['WINNER'] = np.where(R_df['WINNER'] == R_df['FIGHTER'], 1, 0)
R_df.head()

,FIGHTER,KD,SIG_PCT,TD_PCT,SUB_ATT,GROUND_ATT,WIN_TYPE,NUM_RD,TYPE,SIG_STR_SUC,SIG_STR_ATT,TOT_STR_SUC,TOT_STR_ATT,TD_SUC,TD_ATT,GROUND_SUC,TOTAL_TIME,WINNER,LOC,REF,DATE
0,Aljamain Sterling,0,48,0,0,0,KO/TKO,5,UFC Bantamweight Title Bout,17,35,24,46,0,2,0,351.0,0,"Boston, Massachusetts, USA",Marc Goddard,2023-08-19
1,Chris Weidman,0,40,0,0,0,Decision - Unanimous,3,Middleweight Bout,37,92,39,94,0,8,0,900.0,0,"Boston, Massachusetts, USA",Keith Peterson,2023-08-19
2,Neil Magny,0,40,0,0,0,Decision - Unanimous,3,Welterweight Bout,27,66,51,93,0,2,0,900.0,0,"Boston, Massachusetts, USA",Keith Peterson,2023-08-19
3,Marlon Vera,0,56,0,0,1,Decision - Unanimous,3,Bantamweight Bout,141,251,141,251,0,1,0,900.0,1,"Boston, Massachusetts, USA",Herb Dean,2023-08-19
4,Zhang Weili,1,75,85,0,122,Decision - Unanimous,5,UFC Women's Strawweight Title Bout,163,217,296,358,6,7,96,1500.0,1,"Boston, Massachusetts, USA",Kevin MacDonald,2023-08-19


In [281]:
B_subset = ['B_FIGHTER','B_KD','B_SIG_PCT','B_TD_PCT', 'B_SUB_ATT', 'B_GROUND_ATT', 'WIN_TYPE', 'NUM_RD', 'TYPE',
                'B_SIG_STR_SUC', 'B_SIG_STR_ATT', 'B_TOT_STR_SUC', 'B_TOT_STR_ATT', 'B_TD_SUC', 'B_TD_ATT',
                'B_GROUND_SUC', 'TOTAL_TIME','WINNER', 'LOC', 'REF', 'DATE']
B_df = df[B_subset]
B_df = B_df.rename(columns=lambda x: x.replace('B_', ''))
B_df = B_df.rename(columns={'SUATT':'SUB_ATT'})
B_df['WINNER'] = np.where(R_df['WINNER'] == R_df['FIGHTER'], 1, 0)
B_df.head()

,FIGHTER,KD,SIG_PCT,TD_PCT,SUB_ATT,GROUND_ATT,WIN_TYPE,NUM_RD,TYPE,SIG_STR_SUC,SIG_STR_ATT,TOT_STR_SUC,TOT_STR_ATT,TD_SUC,TD_ATT,GROUND_SUC,TOTAL_TIME,WINNER,LOC,REF,DATE
0,Sean O'Malley,1,71,---,0,18,KO/TKO,5,UFC Bantamweight Title Bout,25,35,26,36,0,0,14,351.0,0,"Boston, Massachusetts, USA",Marc Goddard,2023-08-19
1,Brad Tavares,0,60,---,0,0,Decision - Unanimous,3,Middleweight Bout,70,115,71,116,0,0,0,900.0,0,"Boston, Massachusetts, USA",Keith Peterson,2023-08-19
2,Ian Garry,0,60,100,0,5,Decision - Unanimous,3,Welterweight Bout,91,150,111,171,2,2,3,900.0,0,"Boston, Massachusetts, USA",Keith Peterson,2023-08-19
3,Pedro Munhoz,0,45,0,0,0,Decision - Unanimous,3,Bantamweight Bout,113,246,113,246,0,1,0,900.0,0,"Boston, Massachusetts, USA",Herb Dean,2023-08-19
4,Amanda Lemos,0,38,---,2,0,Decision - Unanimous,5,UFC Women's Strawweight Title Bout,24,63,29,68,0,0,0,1500.0,0,"Boston, Massachusetts, USA",Kevin MacDonald,2023-08-19


In [282]:
new_df = pd.concat([R_df, B_df])

# sort the combined dataframe by date
df = new_df
# reset the index
df = df.reset_index(drop=True)
df

,FIGHTER,KD,SIG_PCT,TD_PCT,SUB_ATT,GROUND_ATT,WIN_TYPE,NUM_RD,TYPE,SIG_STR_SUC,SIG_STR_ATT,TOT_STR_SUC,TOT_STR_ATT,TD_SUC,TD_ATT,GROUND_SUC,TOTAL_TIME,WINNER,LOC,REF,DATE
0,Aljamain Sterling,0,48,0,0,0,KO/TKO,5,UFC Bantamweight Title Bout,17,35,24,46,0,2,0,351.0,0,"Boston, Massachusetts, USA",Marc Goddard,2023-08-19
1,Chris Weidman,0,40,0,0,0,Decision - Unanimous,3,Middleweight Bout,37,92,39,94,0,8,0,900.0,0,"Boston, Massachusetts, USA",Keith Peterson,2023-08-19
2,Neil Magny,0,40,0,0,0,Decision - Unanimous,3,Welterweight Bout,27,66,51,93,0,2,0,900.0,0,"Boston, Massachusetts, USA",Keith Peterson,2023-08-19
3,Marlon Vera,0,56,0,0,1,Decision - Unanimous,3,Bantamweight Bout,141,251,141,251,0,1,0,900.0,1,"Boston, Massachusetts, USA",Herb Dean,2023-08-19
4,Zhang Weili,1,75,85,0,122,Decision - Unanimous,5,UFC Women's Strawweight Title Bout,163,217,296,358,6,7,96,1500.0,1,"Boston, Massachusetts, USA",Kevin MacDonald,2023-08-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14031,Steve Berger,0,11,---,3,0,Decision - Unanimous,3,Welterweight Bout,2,18,108,129,0,0,0,900.0,0,"Atlantic City, New Jersey, USA",Mario Yamasaki,2001-05-04
14032,Joey Gilbert,0,42,0,0,0,KO/TKO,3,Lightweight Bout,3,7,11,15,0,1,0,297.0,0,"Atlantic City, New Jersey, USA",John McCarthy,2001-05-04
14033,Matt Serra,0,40,15,3,2,KO/TKO,3,Welterweight Bout,10,25,27,44,2,13,2,891.0,0,"Atlantic City, New Jersey, USA",Mario Yamasaki,2001-05-04
14034,Pete Williams,0,66,25,0,6,KO/TKO,3,Heavyweight Bout,6,9,14,18,1,4,5,388.0,0,"Atlantic City, New Jersey, USA",John McCarthy,2001-05-04


In [283]:
df.replace("---", int(0), inplace=True)

In [284]:
df['TYPE'].unique()

array(['UFC Bantamweight Title Bout', 'Middleweight Bout',
       'Welterweight Bout', 'Bantamweight Bout',
       "UFC Women's Strawweight Title Bout", 'Lightweight Bout',
       "Women's Flyweight Bout", 'Light Heavyweight Bout',
       'Featherweight Bout', "Women's Strawweight Bout",
       'Heavyweight Bout', 'Catch Weight Bout', 'Flyweight Bout',
       "Women's Bantamweight Bout", "Women's Featherweight Bout",
       'UFC Featherweight Title Bout', 'UFC Flyweight Title Bout',
       "UFC Women's Bantamweight Title Bout",
       'UFC Middleweight Title Bout', 'UFC Welterweight Title Bout',
       'UFC Heavyweight Title Bout', "UFC Women's Flyweight Title Bout",
       'UFC Interim Featherweight Title Bout',
       'UFC Lightweight Title Bout', 'UFC Light Heavyweight Title Bout',
       'UFC Interim Flyweight Title Bout',
       'UFC Interim Bantamweight Title Bout',
       'UFC Interim Heavyweight Title Bout',
       "UFC Women's Featherweight Title Bout",
       'UFC Interim Lig

In [285]:
df['TYPE'] = df['TYPE'].str.replace('Bout', '')
df['TYPE'] = df['TYPE'].str.replace('Title', '')
df['TYPE'] = df['TYPE'].str.replace('Tournament', '')
df['TYPE'] = df['TYPE'].str.replace('Ultimate Fighter', '')
df['TYPE'] = df['TYPE'].str.replace('UFC', '')
df['TYPE'] = df['TYPE'].str.replace('Interim', '')
df['TYPE'] = df['TYPE'].str.replace('Brazil', '')
df['TYPE'] = df['TYPE'].str.replace('America', '')
df['TYPE'] = df['TYPE'].str.replace('China', '')
df['TYPE'] = df['TYPE'].str.replace('TUF', '')
df['TYPE'] = df['TYPE'].str.replace('Australia', '')
df['TYPE'] = df['TYPE'].str.replace('Nations', '')
df['TYPE'] = df['TYPE'].str.replace('Canada', '')
df['TYPE'] = df['TYPE'].str.replace('vs.', '')
df['TYPE'] = df['TYPE'].str.replace('UK', '')
df['TYPE'] = df['TYPE'].str.replace('Latin', '')
df['TYPE'] = df['TYPE'].str.replace("Women's", 'W')
df['TYPE'].unique()

array([' Bantamweight  ', 'Middleweight ', 'Welterweight ',
       'Bantamweight ', ' W Strawweight  ', 'Lightweight ',
       'W Flyweight ', 'Light Heavyweight ', 'Featherweight ',
       'W Strawweight ', 'Heavyweight ', 'Catch Weight ', 'Flyweight ',
       'W Bantamweight ', 'W Featherweight ', ' Featherweight  ',
       ' Flyweight  ', ' W Bantamweight  ', ' Middleweight  ',
       ' Welterweight  ', ' Heavyweight  ', ' W Flyweight  ',
       '  Featherweight  ', ' Lightweight  ', ' Light Heavyweight  ',
       '  Flyweight  ', '  Bantamweight  ', '  Heavyweight  ',
       ' W Featherweight  ', '  Lightweight  ', '  Middleweight  ',
       ' 28 Heavyweight   ', ' 28 W Featherweight   ',
       ' 27 Featherweight   ', ' 27 Lightweight   ', '  Welterweight  ',
       ' 25 Welterweight   ', '   3 Lightweight   ',
       ' 23 W Strawweight   ', ' 23 Light Heavyweight   ',
       '  Light Heavyweight  ', ' 22 Lightweight   ',
       '   2 Welterweight   ', '   2 Lightweight   ',
     

In [286]:
for attribute in df.columns:
    if str(df[attribute].iloc[0])[0].isnumeric() and attribute != "DATE":
        # print(str(df[attribute][0])[0])
        # print(attribute)
        df[attribute] = pd.to_numeric(df[attribute]).astype("int16")

In [287]:
from sklearn.preprocessing import OneHotEncoder

attsToEncode = ['REF', 'TYPE', 'DATE','WIN_TYPE', 'LOC', 'FIGHTER']
# for col in df.columns:
#     if df[col].dtypes != np.int16 and col not in ["FIGHTER", "WINNER"]:
#         attsToEncode.append(col)

# print(len(attsToEncode))

In [288]:
from sklearn.preprocessing import LabelEncoder

# Sample DataFrame with 'RED', 'BLUE', and 'WINNER' columns
# df = pd.read_csv("ufc_dataset.csv")  # Load your dataset

# Initialize LabelEncoder
# le = LabelEncoder()

# # Combine all the fighter names from 'RED', 'BLUE', and 'WINNER' columns

# # If the same fighters can appear in both 'RED', 'BLUE', and 'WINNER' columns, you'd concatenate all three
# all_fighters = pd.concat([df['R_FIGHTER'], df['B_FIGHTER'], df['WINNER']]).unique()
# # If 'WINNER' column always contains a subset of what's in 'RED' and 'BLUE', you might exclude it from concatenation
# # all_fighters = pd.concat([df['RED'], df['BLUE']]).unique()

# le.fit(all_fighters)

# # Transform each column reusing the same encoder
# df['RED_encoded'] = le.transform(df['R_FIGHTER'])
# df['BLUE_encoded'] = le.transform(df['B_FIGHTER'])
# df['WINNER_encoded'] = le.transform(df['WINNER'])

# # Now you can drop the original columns if you don't need them
# df.drop(['R_FIGHTER', 'B_FIGHTER', 'WINNER'], axis=1, inplace=True)


# Extract the columns to be encoded
df_to_encode = df[attsToEncode]

# Create a OneHotEncoder instance
encoder = OneHotEncoder()

# Fit and transform the selected columns
encoded_data = encoder.fit_transform(df_to_encode)

# Convert the encoded data to a DataFrame
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(attsToEncode))

# Concatenate the original DataFrame with the encoded DataFrame
df_encoded = pd.concat([df, encoded_df], axis=1)

# Drop the original categorical columns
df_encoded.drop(columns=attsToEncode, inplace=True)
df_encoded

KD  SIG_PCT  TD_PCT  SUB_ATT  GROUND_ATT  NUM_RD  SIG_STR_SUC  \
0       0       48       0        0           0       5           17   
1       0       40       0        0           0       3           37   
2       0       40       0        0           0       3           27   
3       0       56       0        0           1       3          141   
4       1       75      85        0         122       5          163   
...    ..      ...     ...      ...         ...     ...          ...   
14031   0       11       0        3           0       3            2   
14032   0       42       0        0           0       3            3   
14033   0       40      15        3           2       3           10   
14034   0       66      25        0           6       3            6   
14035   1       38       0        0           9       5           42   

       SIG_STR_ATT  TOT_STR_SUC  TOT_STR_ATT  TD_SUC  TD_ATT  GROUND_SUC  \
0               35           24           46       0       2           0   
1               92           39           94       0       8           0   
2               66           51           93       0       2           0   
3              251          141          251       0       1           0   
4              217          296          358       6       7          96   
...            ...          ...          ...     ...     ...         ...   
14031           18          108          129       0       0           0   
14032            7           11           15       0       1           0   
14033           25           27           44       2      13           2   
14034            9           14           18       1       4           5   
14035          108           76          144       0       0           6   

       TOTAL_TIME  WINNER  REF_Adam Cheadle  REF_Adam Martinez  REF_Al Coley  \
0             351       0               0.0                0.0           0.0   
1             900       0               0.0                0.0           0.0   
2             900       0               0.0                0.0           0.0   
3             900       1               0.0                0.0           0.0   
4            1500       1               0.0                0.0           0.0   
...           ...     ...               ...                ...           ...   
14031         900       0               0.0                0.0           0.0   
14032         297       0               0.0                0.0           0.0   
14033         891       0               0.0                0.0           0.0   
14034         388       0               0.0                0.0           0.0   
14035        1500       0               0.0                0.0           0.0   

       REF_Al Guinee  REF_Al Wichgers  REF_Alan Abeles  REF_Anders Ohlsson  \
0                0.0              0.0              0.0                 0.0   
1                0.0              0.0              0.0                 0.0   
2                0.0              0.0              0.0                 0.0   
3                0.0              0.0              0.0                 0.0   
4                0.0              0.0              0.0                 0.0   
...              ...              ...              ...                 ...   
14031            0.0              0.0              0.0                 0.0   
14032            0.0              0.0              0.0                 0.0   
14033            0.0              0.0              0.0                 0.0   
14034            0.0              0.0              0.0                 0.0   
14035            0.0              0.0              0.0                 0.0   

       REF_Andrew Glenn  REF_Andy Social  REF_Anthony Hamlett  \
0                   0.0              0.0                  0.0   
1                   0.0              0.0                  0.0   
2                   0.0              0.0                  0.0   
3                   0.0              0.0                  0.0   
4     

# Decision Tree Classifier

In [289]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Prepare your feature matrix X and target vector y
X = df_encoded.drop(columns=['WINNER'])  # Drop the target column from the DataFrame
y = df_encoded['WINNER']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

# Initialize the decision tree classifier
clf = DecisionTreeClassifier(random_state=41)

# Fit the decision tree classifier
clf.fit(X_train, y_train)

# Evaluate your classifier using the test data
accuracy = clf.score(X_test, y_test)
print(f"Accuracy of the Decision Tree classifier is: {accuracy:.4f}")

Accuracy of the Decision Tree classifier is: 0.7041
